In [18]:
import numpy as np
import pandas as pd
mydata = pd.read_csv('subset.csv')
Total = mydata['is_booking'].sum()
#print Total
#-1 means the hotel_cluster is never clicked
# 0 means the hotel_cluster is clikced(interested) but not booked
# 1 means the hotel_cluster is booked
m = np.zeros((10000,100))
m[m==0]=-1
#Build the dataset Matrix
userid = mydata['user_id'].unique()
hotelcluster = mydata['hotel_cluster'].unique()
for index, row in mydata.iterrows():
    rowNum = np.where(userid==row['user_id'])
    colNum = np.where(hotelcluster==row['hotel_cluster'])
    if m[rowNum,colNum]==-1:
        m[rowNum,colNum]=row['is_booking']+1
    else:
        m[rowNum,colNum] = m[rowNum,colNum]+row['is_booking']
print m.max()

37.0


In [19]:
#############################    Below are analysis for SVD    #####################################
import scipy.sparse as sp
from scipy.sparse.linalg import svds
from math import sqrt

In [20]:
def accuracy(pred_mat,TestIndex,TestMatrix):
    new_mat = np.zeros((10000, 100))
    for index in TestIndex:
        new_mat[index] = pred_mat[index]
    return 1-(np.count_nonzero(new_mat - TestMatrix)/(1.0*len(TestIndex)))

def splitDataSet(percent):
    pMatrix = np.random.rand(10000,100)
    TrainMatrix = np.zeros([10000,100])
    TestMatrix = np.zeros([10000,100])
    TestIndex = []
    for index, x in np.ndenumerate(pMatrix):
        if x >= percent:
            TrainMatrix[index] = m[index]
        else:
            TestIndex.append(index)
            TestMatrix[index] = m[index]
    TrainMatrix[TrainMatrix<0]=0
    TrainMatrix[TrainMatrix>1]=1
    TestMatrix[TestMatrix>=2] = 1
    TestMatrix[TestMatrix<0] = 0
    return TrainMatrix,TestMatrix,TestIndex

In [13]:
#get SVD components from train matrix. Choose k.
accuracy_ls = []
for i in range(1,100):
    cur_accuracy = []
    for j in range(1,6):
        TrainMatrix,TestMatrix,TestIndex = splitDataSet(0.1)
        u, s, vt = svds(TrainMatrix, k = i)
        s_diag_matrix = np.diag(s)
        X_pred = np.dot(np.dot(u, s_diag_matrix), vt)
        # If predicted entry < 0.5, set it to 0. If predicted entry >= 0.5, set it to 1.
        X_pred[X_pred<0.5]=0
        X_pred[X_pred>=0.5]=1
        cur_accuracy.append(accuracy(X_pred,TestIndex,TestMatrix))
    accuracy_ls.append(np.mean(cur_accuracy))

KeyboardInterrupt: 

In [16]:
# Baseline
baseline = []
for i in range(1,11):
    TrainMatrix,TestMatrix,TestIndex = splitDataSet(0.3)
    baseline.append(np.count_nonzero(TestMatrix)/(len(TestIndex)*1.0))
print baseline

[0.12240848329476786, 0.12355541353783885, 0.12299205472524255, 0.12305753464848086, 0.12320981738806368, 0.12363420546351157, 0.12371271855791054, 0.122868784894528, 0.12324793294521051, 0.12257426928605654]


In [17]:
1-np.mean(baseline)

0.87687387852583887

In [26]:
#Combine the predicted entries with the trainMatrix
TrainMatrix,TestMatrix,TestIndex = splitDataSet(0.1)
u, s, vt = svds(TrainMatrix, k = 8)
s_diag_matrix = np.diag(s)
result = np.dot(np.dot(u, s_diag_matrix), vt)
result[result<0.5]=0
result[result>=0.5]=1
print result.max()

#Popular items in the resulted matrix
Rpopularity = np.sum(result, axis=0)
print np.argsort(Rpopularity)[-10:,]

#Coverage of the recommendation based on predicted results
recommendation = np.zeros([10000,5])
for rowNum in range(result.shape[0]):
    #print sum(result[rowNum])
    recommend = np.argsort(result[rowNum])[-5:,]
    #print recomend
    recommendation[rowNum]=recommend
print np.unique(recommendation)
print len(np.unique(recommendation))

#Coverage of the recomendation based on original dataset
Mrecommendation = np.zeros([10000,5])
for rowN in range(m.shape[0]):
    #print sum(m[rowN])
    Mrecommend = np.argsort(m[rowN])[-5:,]
    #print recomend
    Mrecommendation[rowN]=Mrecommend
print np.unique(Mrecommendation)
print len(np.unique(Mrecommendation))

#Count the number of same recommendations given by our result compared to the original matrix
count = 0
for rowNum in range(result.shape[0]):
    recommend = np.argsort(result[rowNum])[-5:,]
    Mrecomend = np.argsort(m[rowN])[-5:,]
    if recommend.all() == Mrecomend.all():
        count+=1
print count

1.0
[58 43 69 37 20 80 62 70  1 49]
[  0.   1.   2.   3.   4.   6.   7.   8.   9.  10.  11.  12.  13.  14.  15.
  16.  17.  18.  19.  20.  21.  22.  23.  24.  25.  26.  27.  28.  29.  30.
  31.  32.  33.  34.  35.  36.  37.  38.  39.  40.  41.  42.  43.  44.  45.
  46.  47.  48.  49.  50.  51.  52.  53.  54.  55.  56.  57.  58.  59.  60.
  61.  62.  63.  64.  65.  66.  67.  68.  69.  70.  71.  72.  73.  74.  75.
  76.  77.  78.  79.  80.  81.  82.  83.  84.  86.  87.  89.  95.  96.  97.
  98.  99.]
92
[  0.   1.   2.   3.   4.   5.   6.   7.   8.   9.  10.  11.  12.  13.  14.
  15.  16.  17.  18.  19.  20.  21.  22.  23.  24.  25.  26.  27.  28.  29.
  30.  31.  32.  33.  34.  35.  36.  37.  38.  39.  40.  41.  42.  43.  44.
  45.  46.  47.  48.  49.  50.  51.  52.  53.  54.  55.  56.  57.  58.  59.
  60.  61.  62.  63.  64.  65.  66.  67.  68.  69.  70.  71.  72.  73.  74.
  75.  76.  77.  78.  79.  80.  81.  82.  83.  84.  85.  86.  87.  88.  89.
  90.  91.  92.  93.  94.  95.  96.  